<a href="https://colab.research.google.com/github/FNAGHAV/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The os module in Python provides a way of using operating system-dependent functionality like reading or writing to the file system.
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
# Set the Spark version
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
#In Google Colab, the installation of Java and Spark needs to be performed each time you start a new session. This is because Google Colab provides a temporary environment, and when you close your Colab notebook or your session times out, the environment is reset.
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set environment variables for Java and Spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:

# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [4]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

from pyspark.sql.functions import year, col, round

# Step 1: Extract the year from the date column
df = df.withColumn("year", year(col("date")))

# Step 2: Filter the DataFrame to include only entries where bedrooms equals four
four_bedroom_df = df.filter(col("bedrooms") == 4)

# Step 3: Ensure "price" column is cast to a numeric type
four_bedroom_df = four_bedroom_df.withColumn("price", four_bedroom_df["price"].cast("float"))

# Step 4: Group the data by the year
grouped_df = four_bedroom_df.groupBy("year")

# Step 5: Calculate the average price for each year
average_price_df = grouped_df.avg("price")

# Step 6: Round the average price to two decimal places
result = average_price_df.withColumn("avg_price_rounded", round("avg(price)", 2))

# Show the result
result.show()

+----+------------------+-----------------+
|year|        avg(price)|avg_price_rounded|
+----+------------------+-----------------+
|2022| 296363.8845050215|        296363.88|
|2019| 300263.6955128205|         300263.7|
|2020|298353.78003169573|        298353.78|
|2021|   301819.44398864|        301819.44|
+----+------------------+-----------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
from pyspark.sql.functions import col, round

# Step 1: Filter the DataFrame to include only entries where bedrooms equals 3 and bathrooms equals 3
three_bedroom_three_bathroom_df = df.filter((col("bedrooms") == 3) & (col("bathrooms") == 3))

# Step 2: Ensure "price" column is cast to a numeric type
three_bedroom_three_bathroom_df = three_bedroom_three_bathroom_df.withColumn("price", three_bedroom_three_bathroom_df["price"].cast("float"))

# Step 3: Group the data by the year the home was built
grouped_df = three_bedroom_three_bathroom_df.groupBy("date_built")

# Step 4: Calculate the average price for each year
average_price_df = grouped_df.avg("price")

# Step 5: Round the average price to two decimal places
result = average_price_df.withColumn("avg_price_rounded", round("avg(price)", 2))

# Show the result
result.show()

+----------+------------------+-----------------+
|date_built|        avg(price)|avg_price_rounded|
+----------+------------------+-----------------+
|      2016|  290555.073964497|        290555.07|
|      2012| 293683.1872074883|        293683.19|
|      2017| 292676.7887740029|        292676.79|
|      2014| 290852.2661870504|        290852.27|
|      2013|295962.27145085804|        295962.27|
|      2011|291117.46706586826|        291117.47|
|      2015| 288770.2966101695|         288770.3|
|      2010|  292859.615942029|        292859.62|
+----------+------------------+-----------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
from pyspark.sql.functions import col, round

# Step 1: Filter the DataFrame to include only entries meeting the specified criteria
filtered_df = df.filter((col("bedrooms") == 3) &
                        (col("bathrooms") == 3) &
                        (col("floors") == 2) &
                        (col("sqft_living") >= 2000))

# Step 2: Ensure "price" column is cast to a numeric type
filtered_df = filtered_df.withColumn("price", filtered_df["price"].cast("float"))

# Step 3: Group the data by the year the home was built
grouped_df = filtered_df.groupBy("date_built")

# Step 4: Calculate the average price for each year
average_price_df = grouped_df.avg("price")

# Step 5: Round the average price to two decimal places
result = average_price_df.withColumn("avg_price_rounded", round("avg(price)", 2))

# Show the result
result.show()


+----------+------------------+-----------------+
|date_built|        avg(price)|avg_price_rounded|
+----------+------------------+-----------------+
|      2016| 293965.1046511628|         293965.1|
|      2012|307539.97402597405|        307539.97|
|      2017|280317.57692307694|        280317.58|
|      2014| 298264.7183908046|        298264.72|
|      2013|      303676.79375|        303676.79|
|      2011| 276553.8128654971|        276553.81|
|      2015| 297609.9679144385|        297609.97|
|      2010| 285010.2215909091|        285010.22|
+----------+------------------+-----------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
from pyspark.sql.functions import round, avg

# Step 1: Calculate the average price of a home per "view" rating
average_price_per_view_df = df.groupBy("view").agg(round(avg("price"), 2).alias("avg_price"))

# Step 2: Filter the results to include only those with an average home price greater than or equal to $350,000
filtered_df = average_price_per_view_df.filter(col("avg_price") >= 350000)

# Step 3: Order the results by descending view rating
result = filtered_df.orderBy("view", ascending=False)

# Show the result
result.show()

# Step 4: Determine the runtime for this query
start_time = time.time()

# Perform the query
result.collect()

end_time = time.time()
runtime = end_time - start_time
print("Runtime for the query:", runtime, "seconds")
start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

Runtime for the query: 1.6537797451019287 seconds
--- 8.249282836914062e-05 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
# Cache the DataFrame
df.cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, year: int]

In [10]:
# 8. Check if the table is cached.
if df.is_cached:
    print("The DataFrame representing the temporary table home_sales is cached.")
else:
    print("The DataFrame representing the temporary table home_sales is not cached.")

The DataFrame representing the temporary table home_sales is cached.


In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

from pyspark.sql.functions import round, avg
import time

# Step 1: Calculate the average price of a home per "view" rating
average_price_per_view_df = df.groupBy("view").agg(round(avg("price"), 2).alias("avg_price"))

# Step 2: Filter the results to include only those with an average home price greater than or equal to $350,000
filtered_df = average_price_per_view_df.filter(col("avg_price") >= 350000)

# Step 3: Order the results by descending view rating
result_cached = filtered_df.orderBy("view", ascending=False)

# Show the result
result_cached.show()

# Step 4: Determine the runtime for this query using cached data
start_time_cached = time.time()

# Perform the query
result_cached.collect()

end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached
print("Runtime for the query using cached data:", runtime_cached, "seconds")

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

Runtime for the query using cached data: 0.7243607044219971 seconds
--- 6.031990051269531e-05 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("partitioned_home_sales.parquet")


In [14]:
# 11. Read the parquet formatted data.
# Read the partitioned Parquet data
partitioned_df = spark.read.parquet("partitioned_home_sales.parquet")

# Show the schema of the DataFrame
partitioned_df.printSchema()

# Show the first few rows of the DataFrame
partitioned_df.show()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       

In [15]:
# 12. Create a temporary table for the parquet data.
# Register the DataFrame as a temporary table using SQLContext
try:
    partitioned_df.createOrReplaceTempView("partitioned_home_sales_view")
    print("Temporary view 'partitioned_home_sales_view' created successfully.")
except Exception as e:
    print("Failed to create temporary view. Error:", str(e))

Temporary view 'partitioned_home_sales_view' created successfully.


In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

from pyspark.sql.functions import round, avg
import time

# Step 1: Calculate the average price of a home per "view" rating
average_price_per_view_df = partitioned_df.groupBy("view").agg(round(avg("price"), 2).alias("avg_price"))

# Step 2: Filter the results to include only those with an average home price greater than or equal to $350,000
filtered_df = average_price_per_view_df.filter(col("avg_price") >= 350000)

# Step 3: Order the results by descending view rating
result_parquet = filtered_df.orderBy("view", ascending=False)

# Show the result
result_parquet.show()

# Step 4: Determine the runtime for this query using Parquet DataFrame
start_time_parquet = time.time()

# Perform the query
result_parquet.collect()

end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet
print("Runtime for the query using Parquet DataFrame:", runtime_parquet, "seconds")

# Compare with cached runtime
print("Cached runtime:", runtime_cached, "seconds")
start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

Runtime for the query using Parquet DataFrame: 1.1259207725524902 seconds
Cached runtime: 0.7243607044219971 seconds
--- 9.608268737792969e-05 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
df.unpersist()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, year: int]

In [18]:
# 15. Check if the home_sales is no longer cached

if not df.is_cached:
    print("The DataFrame representing the temporary table home_sales is no longer cached.")
else:
    print("The DataFrame representing the temporary table home_sales is still cached.")

The DataFrame representing the temporary table home_sales is no longer cached.
